In [1]:
#UNADJUSTED, ADJUSTED, AND ALL RATIOS MUST BE PLOTTED IN ORDER

import uproot
import awkward as ak
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter
from IPython.display import Image, display
from IPython.display import HTML
import threading

np.seterr(divide='ignore', invalid='ignore') # Ignore division errors

# Use pandas to make an array of values ordered by event ID
features=['EventWeight', 'phi', 'eta', 'probeCharge']
for i in range(4):
    features.append('dtDRbyStation_'+str(i))
    features.append('dtDPhibyStation_'+str(i))
    features.append('dtDEtabyStation_'+str(i))

class Type:
    def __init__(self, name, features):
        self.name = name
        self.sameSignFile = uproot.open('highHcal_'+str(name)+'_tagAligned (2).root')
        self.sameSignEvents = self.sameSignFile['demo']['allEvents']['sigVariables']
        self.branches = self.sameSignEvents.arrays()
        self.EventWeight = self.branches['EventWeight']
        self.eta = self.branches['eta']
        self.phi = self.branches['phi']

        self.Events = self.sameSignEvents.arrays(features, library='pd')
        
        
        self.DR_S = []
        self.DPhi_S = []
        self.DEta_S = []

        for i in range (4):
            self.DR_S.append(self.branches['dtDRbyStation_' + str(i)])
            self.DPhi_S.append(self.branches['dtDPhibyStation_' + str(i)])
            self.DEta_S.append(self.branches['dtDEtabyStation_' + str(i)])

        self.proximity_mask = []

Data = Type('data', features)
MC = Type('mc', features)

print('init uproot')

#Data.sameSignFile.keys()
#Data.sameSignEvents
Data.sameSignEvents.keys()
#Data.branches['pt']

init uproot


['pt',
 'eta',
 'phi',
 'staDR',
 'staPhi',
 'staE',
 'standaloneDEoverE',
 'staChi',
 'cscDR',
 'probeCharge',
 'ecalIso',
 'cellEdgeDeta',
 'cellEdgeDphi',
 'PUupWeight',
 'PUdownWeight',
 'IDupWeight',
 'IDdownWeight',
 'ISOupWeight',
 'ISOdownWeight',
 'TrigUpWeight',
 'TrigDownWeight',
 'EnBinWeight',
 'lumi',
 'event',
 'cscDRbyStation_0',
 'dtDRbyStation_0',
 'cscDEtabyStation_0',
 'dtDEtabyStation_0',
 'cscDPhibyStation_0',
 'dtDPhibyStation_0',
 'cscDZbyStation_0',
 'dtDZbyStation_0',
 'cscDRbyStation_1',
 'dtDRbyStation_1',
 'cscDEtabyStation_1',
 'dtDEtabyStation_1',
 'cscDPhibyStation_1',
 'dtDPhibyStation_1',
 'cscDZbyStation_1',
 'dtDZbyStation_1',
 'cscDRbyStation_2',
 'dtDRbyStation_2',
 'cscDEtabyStation_2',
 'dtDEtabyStation_2',
 'cscDPhibyStation_2',
 'dtDPhibyStation_2',
 'cscDZbyStation_2',
 'dtDZbyStation_2',
 'cscDRbyStation_3',
 'dtDRbyStation_3',
 'cscDEtabyStation_3',
 'dtDEtabyStation_3',
 'cscDPhibyStation_3',
 'dtDPhibyStation_3',
 'cscDZbyStation_3',
 'dtD

In [2]:
#Coord refers to DR, DPhi, or DEta
class Coord:
    def __init__(self,name,x1,x2,bins):
        self.name = name
        self.x1 = x1
        self.x2 = x2
        self.bins = np.linspace(x1, x2, bins+1)  # creates an array of bin edges
        self.bin_width = (x2 - x1) / bins

DR = Coord('DR',-1,0.3,300)     # set delta coord edges/bin
DPhi = Coord('DPhi',-3,3,200)   # set delta coord edges/bin
DEta = Coord('DEta',-3,3,150)   # set delta coord edges/bin

mc_eta_mask = abs(MC.Events['eta']) <= 0.85
data_eta_mask = abs(Data.Events['eta']) <= 0.85

MC.Events = MC.Events.loc[mc_eta_mask]
Data.Events = Data.Events.loc[data_eta_mask]

mc_pos_charge_mask = MC.Events['probeCharge'] > 0
mc_neg_charge_mask = MC.Events['probeCharge'] < 0
data_pos_charge_mask = Data.Events['probeCharge'] > 0
data_neg_charge_mask = Data.Events['probeCharge'] < 0

# Re-bin all points above 0.3 to -1
for i in range(4):
    MC.Events['dtDRbyStation_'+str(i)] = MC.Events['dtDRbyStation_'+str(i)].apply(lambda x: -1 if x > DR.x2 else x)
    Data.Events['dtDRbyStation_'+str(i)] = Data.Events['dtDRbyStation_'+str(i)].apply(lambda x: -1 if x > DR.x2 else x)
    MC.DR_S[i] = ak.Array(MC.Events['dtDRbyStation_'+str(i)])
    Data.DR_S[i] = ak.Array(Data.Events['dtDRbyStation_'+str(i)])
    
    MC.Events['dtDPhibyStation_'+str(i)] = MC.Events['dtDPhibyStation_'+str(i)].apply(lambda x: -1 if abs(x) > DPhi.x2 else x) 
    Data.Events['dtDPhibyStation_'+str(i)] = Data.Events['dtDPhibyStation_'+str(i)].apply(lambda x: -1 if abs(x) > DPhi.x2 else x) 
    MC.DPhi_S[i] = ak.Array(MC.Events['dtDPhibyStation_'+str(i)])
    Data.DPhi_S[i] = ak.Array(Data.Events['dtDPhibyStation_'+str(i)])
    
    MC.Events['dtDEtabyStation_'+str(i)] = MC.Events['dtDEtabyStation_'+str(i)].apply(lambda x: -1 if abs(x) > DEta.x2 else x) 
    Data.Events['dtEtabyStation_'+str(i)] = Data.Events['dtDEtabyStation_'+str(i)].apply(lambda x: -1 if abs(x) > DEta.x2 else x) 
    MC.DEta_S[i] = ak.Array(MC.Events['dtDEtabyStation_'+str(i)]) 
    Data.DEta_S[i] = ak.Array(Data.Events['dtDEtabyStation_'+str(i)])

print('Re-Binned points')

#---------------------------------------------------------------------------------------------------------------------------------------

Coordinates = [DR, DPhi, DEta]
for i in range(4):
    for coord in Coordinates:
        bin_indices = np.zeros_like(MC.DR_S[i].to_numpy())                      # Array for storing bin indices
        for j, deltaCoord in enumerate(MC.DR_S[i]):                             # [(0, delta_R), (1, delta_R) ...] <--- list(enumerate(")) should give this
            bin_indices[j] = np.digitize(deltaCoord, coord.bins) - 1            # returns bin index for each delta coord
            MC.Events[str(coord.name)+'_Bin_Station_'+str(i)] = bin_indices      # Add new column with bin indices
        print('MC init '+str(coord.name)+'_Bin_Station_'+str(i))
        
def init_data(i):
    for coord in Coordinates:
        bin_indices = np.zeros_like(Data.DR_S[i].to_numpy())                      # Array for storing bin indices
        for j, deltaCoord in enumerate(Data.DR_S[i]):                             # [(0, delta_R), (1, delta_R) ...] <--- list(enumerate(")) should give this
            bin_indices[j] = np.digitize(deltaCoord, coord.bins) - 1            # returns bin index for each delta coord
            Data.Events[str(coord.name)+'_Bin_Station_'+str(i)] = bin_indices      # Add new column with bin indices      
        print('Data init '+str(coord.name)+'_Bin_Station_'+str(i))
    #print(mcEvents[:5])
    
t = []
if __name__ =="__main__": ########################################### if your computer is having a stroke its bc of the multithreading, just match init_data code to the MC code block above #######################################################
    for i in range(4):
        t.append(threading.Thread(target=init_data, args=(i,)))
    for i in range(4):
        t[i].start()
    for i in range(4):
        t[i].join()

print('init dataframe')

# Initialize class to help calculate normalized histograms for any data set
class NormalizedHist:
    def __init__(self, data, view_bins, bin_width):
        self.data = data[0]
        self.view_bins = view_bins
        self.bin_width = bin_width
        self.norm_factor = 1 / (np.sum(self.data) * self.bin_width)              # normalization factor
        self.temp = (data[1] + self.bin_width / 2)                               # offset bin edge values by + 1/2 bin
        self.bin_centers = self.temp[:-1]                                        # cut last value to give bin center
        self.freq = self.data * self.norm_factor                                 # normalized frequency
        self.err = (self.data ** 0.5) * self.norm_factor                         # normalized vertical error bar

print('init classes')

Re-Binned points
MC init DR_Bin_Station_0
MC init DPhi_Bin_Station_0
MC init DEta_Bin_Station_0
MC init DR_Bin_Station_1
MC init DPhi_Bin_Station_1
MC init DEta_Bin_Station_1
MC init DR_Bin_Station_2
MC init DPhi_Bin_Station_2
MC init DEta_Bin_Station_2
MC init DR_Bin_Station_3
MC init DPhi_Bin_Station_3
MC init DEta_Bin_Station_3
Data init DR_Bin_Station_2
Data init DR_Bin_Station_1
Data init DR_Bin_Station_3
Data init DR_Bin_Station_0
Data init DPhi_Bin_Station_1
Data init DPhi_Bin_Station_0
Data init DPhi_Bin_Station_3
Data init DPhi_Bin_Station_2
Data init DEta_Bin_Station_1
Data init DEta_Bin_Station_0
Data init DEta_Bin_Station_3
Data init DEta_Bin_Station_2
init dataframe
init classes


In [3]:
# Method for plotting histograms
def generate_hist(condition, coord, charge, x1, x2, i, j, data):

    charge_ = '' 
    charge_ = '(+) ' if charge == '+' else charge_
    charge_ = '(-) ' if charge == '-' else charge_

    if i == 0 and condition == 'unadjusted':

        ratio_Station, ratio_Iter = data
        
        if charge == '':
            dataEvents = Data.Events.copy()
            mcEvents = MC.Events.copy()
        elif charge == '+':
            dataEvents = Data.Events.loc[data_pos_charge_mask].copy()
            mcEvents = MC.Events.loc[mc_pos_charge_mask].copy()
        elif charge == '-':
            dataEvents = Data.Events.loc[data_neg_charge_mask].copy()
            mcEvents = MC.Events.loc[mc_neg_charge_mask].copy()
    else:
        ratio_Station, ratio_Iter, dataEvents, mcEvents = data

    Data.DR_S[i] = ak.Array(dataEvents['dtDRbyStation_'+str(i)])
    MC.DR_S[i] = ak.Array(mcEvents['dtDRbyStation_'+str(i)])
    Data.DPhi_S[i] = ak.Array(dataEvents['dtDPhibyStation_'+str(i)])
    MC.DPhi_S[i] = ak.Array(mcEvents['dtDPhibyStation_'+str(i)])
    Data.DEta_S[i] = ak.Array(dataEvents['dtDEtabyStation_'+str(i)])
    MC.DEta_S[i] = ak.Array(mcEvents['dtDEtabyStation_'+str(i)]) 
    MC.EventWeight = ak.Array(mcEvents['EventWeight'])
    
    if coord.name == 'DR':
        station_Data = Data.DR_S[i]
        station_MC = MC.DR_S[i]
    elif coord.name == 'DPhi':
        if charge == '':
            station_Data = np.abs(Data.DPhi_S[i])
            station_MC = np.abs(MC.DPhi_S[i])
        else:
            station_Data = Data.DPhi_S[i]
            station_MC = MC.DPhi_S[i]
    elif coord.name == 'DEta':
        if charge == '':
            station_Data = np.abs(Data.DEta_S[i])
            station_MC = np.abs(MC.DEta_S[i]) 
        else: 
            station_Data = Data.DEta_S[i]
            station_MC = MC.DEta_S[i] 

    if condition == 'unadjusted':
        weights = MC.EventWeight
    elif condition == 'adjusted':
        weights = MC.EventWeight*mcEvents['Ratio_Station_'+str(j)].to_list()
    elif condition == 'all ratios':
        if i == 0:
            mcEvents['All_Ratios'] = mcEvents['Ratio_Station_0']*mcEvents['Ratio_Station_1']*mcEvents['Ratio_Station_2']*mcEvents['Ratio_Station_3'] # by event
        weights = MC.EventWeight*mcEvents['All_Ratios'].to_list()
    elif condition == 'iterative':
        if j == 0 and i == 0:
            mcEvents['Ratio_Iter_0'] = mcEvents['Ratio_Station_0'].to_list()
        weights = MC.EventWeight
        for k in range(j+1):
            weights = weights*mcEvents['Ratio_Iter_'+str(k)].to_list()

    # Create numpy histogram data for Data & MC so ratio can be calculated by initializing each as NormalizedHist Class
    data_hist = NormalizedHist(np.histogram(station_Data, bins=coord.bins, range=(coord.x1, coord.x2), density=None), len(coord.bins), coord.bin_width)
    mc_hist = NormalizedHist(np.histogram(station_MC, bins=coord.bins, range=(coord.x1, coord.x2), density=None, weights=weights), len(coord.bins), coord.bin_width)
        
    if condition == 'adjusted' or condition == 'all ratios':
        mc_hist_unadj = NormalizedHist(np.histogram(station_MC, bins=coord.bins, range=(coord.x1, coord.x2), density=None, weights=MC.EventWeight), len(coord.bins), coord.bin_width)
        adj_factor = np.divide(mc_hist.freq, mc_hist_unadj.freq + 1e-10)
    elif condition == 'iterative':
        if j == 0:
            mc_hist_unadj = NormalizedHist(np.histogram(station_MC, bins=coord.bins, range=(coord.x1, coord.x2), density=None, weights=MC.EventWeight), len(coord.bins), coord.bin_width)
        else:
            mc_hist_unadj = NormalizedHist(np.histogram(station_MC, bins=coord.bins, range=(coord.x1, coord.x2), density=None, weights=MC.EventWeight*mcEvents['Ratio_Iter_'+str(j-1)].to_list()), len(coord.bins), coord.bin_width)
        adj_factor = np.divide(mc_hist.freq, mc_hist_unadj.freq + 1e-10)
    
    ratio_freq = np.divide(data_hist.freq, mc_hist.freq)
    ratio_freq = [1 if (math.isinf(x) or math.isnan(x)) else x for x in ratio_freq] # goodify the infinities
    ratio_err = ratio_freq * np.sqrt((data_hist.err / (data_hist.freq + 1e-10))**2 + (mc_hist.err / (mc_hist.freq + 1e-10))**2) 
    
    # calculate ratio error -> propagation of uncertainty

    if condition == 'unadjusted':
        # Loop over events, assign bin ratio to each event
        ratio_Station.append(np.zeros_like(station_MC.to_numpy()))             # create empty array similar to DCoord for station[j]
        for k, deltaCoord in enumerate(station_MC.to_numpy()):                 # [(0, delta_R), (1, delta_R) ...] <--- list(enumerate(")) should give this
            bin_index = np.digitize(deltaCoord, coord.bins) - 1                # returns bin index for each delta coord
            if bin_index >= len(ratio_freq):                                   # check if index is out of bound
                bin_index = len(ratio_freq) - 1                                # assign the last valid index
            ratio_Station[i][k] = ratio_freq[bin_index]                        # find item k in array and change it to the ratio_freq of that bin index    
        assert len(station_MC) == len(ratio_Station[i])            # make sure length matches
        mcEvents['Ratio_Station_'+str(i)] = ratio_Station[i]       # add new column
        ######### print(mcEvents['Ratio_Station_'+str(i)])

    if condition == 'iterative' and i-1 == j:                                  # take ratio freq between Station_i Data and MC adjusted by ratio from station (i-1) adjusted MC for 
        ratio_iter = ratio_freq                                                # recall that ratio_freq has already been iterated in this case
        # Loop over events, assign bin ratio to each event
        ratio_Iter.append(np.zeros_like(station_MC.to_numpy()))                # create empty array similar to DCoord for station[j]
        for k, deltaCoord in enumerate(station_MC.to_numpy()):                 # [(0, delta_R), (1, delta_R) ...] <--- list(enumerate(")) should give this
            bin_index = np.digitize(deltaCoord, coord.bins) - 1                # returns bin index for each delta coord
            if bin_index >= len(ratio_iter):                                   # check if index is out of bound
                bin_index = len(ratio_iter) - 1                                # assign the last valid index
            ratio_Iter[j][k] = ratio_iter[bin_index]                           # find item k in array and change it to the ratio_iter of that bin index
    
        assert len(station_MC) == len(ratio_Iter[j])            # make sure length matches
        mcEvents['Ratio_Iter_'+str(i)] = ratio_Iter[j]          # add new column


        

    #PLOT -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -

    # Adjusting axs[0].step from bin edge to bin center results in loss of first bar because it is shifted 1/2 bin width; this adds that point back at 0
    step_bc = mc_hist.bin_centers+coord.bin_width/2
    step_f = mc_hist.freq
    step_bc.to_numpy
    step_f.to_numpy
    step_bin_centers = np.insert(step_bc, 0, 0.0)
    step_freq = np.insert(step_f, 0, mc_hist.freq[0])

    # Set up histogram
    fig = plt.figure(figsize=(7,5))
    gs = gridspec.GridSpec(2, 2, height_ratios=[3, 1], width_ratios=[1, 19])

    # Plot data as points, use bars for MC
    ax0 = plt.subplot(gs[0, 1])
    if condition == 'unadjusted':
        ax0.set_title(charge_+'Distance to Station '+str(i))
    elif condition == 'adjusted':
        ax0.set_title(charge_+'Distance to Station '+str(i)+' Weighted by Station '+str(j)+' Ratios')
    elif condition == 'all ratios':
        ax0.set_title(charge_+'Distance to Station '+str(i)+' Weighted by All Ratios')
    elif condition == 'iterative':
        ax0.set_title(charge_+'Distance to Station '+str(i)+' Weighted by Iterative Ratios '+str(j))
    ax0.set_yscale("log")
    ax0.set_xlim(x1, x2)
    ax0.step(step_bin_centers, step_freq, label='Monte Carlo')
    ax0.scatter(data_hist.bin_centers, data_hist.freq, s=10, color='darkorange', label='Data')
    ax0.errorbar(data_hist.bin_centers, data_hist.freq, xerr=coord.bin_width/2, yerr=mc_hist.err, fmt='+')
    plt.setp(ax0.get_xticklabels(), visible=False) # remove x ticks from top right
    plt.setp(ax0.get_yticklabels(), visible=False) # remove y ticks from top right
    ax0.legend()
    # Show mean and std for MC
    if charge == '' and (coord.name == 'DEta' or coord.name == 'DPhi') :
        textstr = '\n'.join((
        r'$\mathrm{MC \ mean}=%.4f$' % (np.mean(station_MC[station_MC != 1]), ),
        r'$\mathrm{MC \ }\sigma=%.4f$' % (np.std(station_MC[station_MC != 1]), )))
    else:
        textstr = '\n'.join((
        r'$\mathrm{MC \ mean}=%.4f$' % (np.mean(station_MC[station_MC != -1]), ),
        r'$\mathrm{MC \ }\sigma=%.4f$' % (np.std(station_MC[station_MC != -1]), )))
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax0.text(0.72, 0.78, textstr, transform=ax0.transAxes, fontsize=10,
        verticalalignment='top', bbox=props) 

    
    # Scatter ratio plot
    ax1 = plt.subplot(gs[1, 1], sharex = ax0)
    ax1.scatter(data_hist.bin_centers, ratio_freq, s=10)
    ax1.errorbar(data_hist.bin_centers, ratio_freq, yerr=ratio_err, fmt='+')
    ax1.set_ylim(0,2)
    ax1.set_xlabel('$\Delta$'+coord.name[1:]+' to Station '+str(i))
    plt.setp(ax1.get_yticklabels(), visible=False) # remove y ticks from bottom right

    # Plot underflow bins    
    ax2 = plt.subplot(gs[0, 0], sharey=ax0)
    ax2.step(step_bin_centers-coord.bin_width, step_freq, label='Monte Carlo')
    ax2.scatter(data_hist.bin_centers, data_hist.freq, s=10, color='darkorange', label='Data')
    ax2.errorbar(data_hist.bin_centers, data_hist.freq, xerr=coord.bin_width/2, yerr=mc_hist.err, fmt='+')
    ax2.set_xlim(-1, -1+coord.bin_width)
    ax2.set_ylabel('Event Frequency (Normalized)')
    plt.setp(ax2.get_xticklabels(), visible=False) # remove x ticks from top left

    # Plot underflow ratio
    ax3 = plt.subplot(gs[1, 0], sharey=ax1, sharex=ax2)
    ax3.scatter(data_hist.bin_centers, ratio_freq, s=10)
    ax3.errorbar(data_hist.bin_centers, ratio_freq, yerr=ratio_err, fmt='+')
    ax3.set_ylim(0,2)
    ax3.set_ylabel('Ratio')
    # Show only the first tick label for the bottom left subplot as integer
    xticks = ax3.xaxis.get_major_ticks()
    plt.setp(ax3.get_xticklabels(), visible=False)
    xticks[0].label1.set_visible(True)
    ax3.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
    ax3.tick_params(axis='x', length=0)

    # Remove vertical gap between subplots
    plt.subplots_adjust(wspace=0.0, hspace=0.0)

    # Save plots as .jpg
    if condition == 'unadjusted':
        plt.savefig(charge_+coord.name+' Distance to Station '+str(i)+'.jpg')
    elif condition == 'adjusted':
        plt.savefig(charge_+coord.name+' Distance to Station '+str(i)+' Weighted by Station '+str(j)+' Ratios.jpg')
    elif condition == 'all ratios':
        plt.savefig(charge_+coord.name+' Distance to Station '+str(i)+' Weighted by All Ratios.jpg')
    elif condition == 'iterative':
        plt.savefig(charge_+coord.name+' Distance to Station '+str(i)+' Weighted by Iterative Ratios '+str(j)+'.jpg')
    plt.close() # prevents figure from being displayed when code cell is executed

    # Plot ratios
    if condition != 'unadjusted':
        fig, axs = plt.subplots(1, 2, sharey=True, gridspec_kw={'wspace': 0, 'width_ratios': [1, 19]}, figsize=(7, 5))
        axs[0].scatter(data_hist.bin_centers, adj_factor)
        axs[0].set_xlim(-1,-1+coord.bin_width)
        axs[1].scatter(data_hist.bin_centers, adj_factor)
        axs[1].set_xlim(0,x2)
        plt.xlabel('$\Delta$'+coord.name[1:]+' to Station '+str(j))
        axs[0].set_ylabel('Adjustment Factor')
        # Show only the first tick label for the left subplot as integer
        xticks = axs[0].xaxis.get_major_ticks()
        plt.setp(axs[0].get_xticklabels(), visible=False)
        xticks[0].label1.set_visible(True)
        axs[0].xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
        axs[0].tick_params(axis='x', length=0)
        if condition == 'adjusted':
            plt.title(charge_+'Net Adjustment by Station '+str(j)+' Ratios for Station '+str(i))
            plt.savefig(charge_+coord.name+' Net Adjustment by Station '+str(j)+' Ratios for Station '+str(i)+'.jpg')
        elif condition == 'all ratios':
            plt.title(charge_+'Net Adjustment by All Ratios for Station '+str(i))
            plt.savefig(charge_+coord.name+' Net Adjustment by All Ratios for Station '+str(i)+'.jpg')
        elif condition == 'iterative':
            plt.title(charge_+'Net Adjustment by Iterative Ratios '+str(j)+' for Station '+str(i))
            plt.savefig(charge_+coord.name+' Net Adjustment by Iterative Ratios '+str(j)+' for Station '+str(i)+'.jpg')
        plt.close()

    return ratio_Station, ratio_Iter, dataEvents, mcEvents

print('init generate_hist')
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    
def compile_hist(coord, charge, x1, x2):
    ratio_Station = []
    ratio_Iter = []
    data = ratio_Station, ratio_Iter
    for i in range (4):
        data = generate_hist(condition='unadjusted', coord=coord, charge=charge, x1=x1, x2=x2, i=i, j=0, data=data)
    print('compiled unadjusted')

    for j in range (4):                          #j iterates through station weights
        for i in range (4):                      #i iterates through stations
            data = generate_hist(condition='adjusted', coord=coord, charge=charge, x1=x1, x2=x2, i=i, j=j, data=data)
    print('compiled adjusted')

    for i in range (4):                    
        data = generate_hist(condition='all ratios', coord=coord, charge=charge, x1=x1, x2=x2, i=i, j=0, data=data)
    print('compiled all ratios')

    for j in range (4):                          
        for i in range (4):                      
            data = generate_hist(condition='iterative', coord=coord, charge=charge, x1=x1, x2=x2, i=i, j=j, data=data)
    print('compiled iterative')

print('init compile_hist')

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def plot_hist(coord, charge):

    charge_ = '' 
    charge_ = '(+) ' if charge == '+' else charge_
    charge_ = '(-) ' if charge == '-' else charge_

    for j in range(4):
        for k in range(2):
            print("\n") 
        print('                            ------------------------------------------------------------------------------- All Stations Weighted by Station '+str(j)+' Ratios: -------------------------------------------------------------------------------')
        for k in range(2):
            print("\n")
        for i in range(4):
            display(HTML("<table><tr><td><img src='"+charge_+""+coord.name+" Distance to Station "+str(i)+".jpg'></td><td><img src='"+charge_+""+coord.name+" Net Adjustment by Station "+str(j)+" Ratios for Station "+str(i)+".jpg'></td><td><img src='"+charge_+""+coord.name+" Distance to Station "+str(i)+" Weighted by Station "+str(j)+" Ratios.jpg'></td></tr></table>"))
    
    for k in range(2):
        print("\n") 
    print('                            ------------------------------------------------------------------------------- All Stations Weighted by All Ratios: -------------------------------------------------------------------------------')
    for k in range(2):
        print("\n")
    for i in range(4):
        display(HTML("<table><tr><td><img src='"+charge_+""+coord.name+" Distance to Station "+str(i)+".jpg'></td><td><img src='"+charge_+""+coord.name+" Net Adjustment by All Ratios for Station "+str(i)+".jpg'></td><td><img src='"+charge_+""+coord.name+" Distance to Station "+str(i)+" Weighted by All Ratios.jpg'></td></tr></table>"))

    for j in range(4):
        for k in range(2):
            print("\n") 
        print('                            ------------------------------------------------------------------------------- All Stations Weighted by Iterative Ratios '+str(j)+': -------------------------------------------------------------------------------')
        for k in range(2):
            print("\n")
        for i in range(4):
            display(HTML("<table><tr><td><img src='"+charge_+""+coord.name+" Distance to Station "+str(i)+".jpg'></td><td><img src='"+charge_+""+coord.name+" Net Adjustment by Iterative Ratios "+str(j)+" for Station "+str(i)+".jpg'></td><td><img src='"+charge_+""+coord.name+" Distance to Station "+str(i)+" Weighted by Iterative Ratios "+str(j)+".jpg'></td></tr></table>"))

print('init plot_hist')

print('initialized')

init generate_hist
init compile_hist
init plot_hist
initialized


In [4]:
compile_hist(DR, '', 0, DR.x2)
plot_hist(DR, '')

compiled unadjusted
compiled adjusted
compiled all ratios
compiled iterative




                            ------------------------------------------------------------------------------- All Stations Weighted by Station 0 Ratios: -------------------------------------------------------------------------------






,,


,,


,,


,,


...





                            ------------------------------------------------------------------------------- All Stations Weighted by Iterative Ratios 3: -------------------------------------------------------------------------------






,,


,,


,,


,,


In [5]:
compile_hist(DPhi, '+', -1.5 , 1.5)
plot_hist(DPhi, '+')

compiled unadjusted
compiled adjusted
compiled all ratios
compiled iterative




                            ------------------------------------------------------------------------------- All Stations Weighted by Station 0 Ratios: -------------------------------------------------------------------------------






,,


,,


,,


,,


...





                            ------------------------------------------------------------------------------- All Stations Weighted by Iterative Ratios 3: -------------------------------------------------------------------------------






,,


,,


,,


,,


In [6]:
compile_hist(DEta, '', 0, 1.5)
plot_hist(DEta, '')

compiled unadjusted
compiled adjusted
compiled all ratios
compiled iterative




                            ------------------------------------------------------------------------------- All Stations Weighted by Station 0 Ratios: -------------------------------------------------------------------------------






,,


,,


,,


,,


...





                            ------------------------------------------------------------------------------- All Stations Weighted by Iterative Ratios 3: -------------------------------------------------------------------------------






,,


,,


,,


,,


In [7]:
peak_lim = 0.05

Data.proximity_mask = []
MC.proximity_mask = []
for i in range(4):
    Data.proximity_mask.append(abs(dataEvents['dtDEtabyStation_'+str(i)]) <= peak_lim)
    MC.proximity_mask.append(abs(mcEvents['dtDEtabyStation_'+str(i)]) <= peak_lim)

    
print('In peak MC events: ', np.sum(MC.proximity_mask[0]), np.sum(MC.proximity_mask[1]), np.sum(MC.proximity_mask[2]), np.sum(MC.proximity_mask[3]))
print('Out of peak MC events: ', np.sum(~MC.proximity_mask[0]), np.sum(~MC.proximity_mask[1]), np.sum(~MC.proximity_mask[2]), np.sum(~MC.proximity_mask[3]))

print('In peak Data events: ', np.sum(Data.proximity_mask[0]), np.sum(Data.proximity_mask[1]), np.sum(Data.proximity_mask[2]), np.sum(Data.proximity_mask[3]))
print('Out of peak Data events: ', np.sum(~Data.proximity_mask[0]), np.sum(~Data.proximity_mask[1]), np.sum(~Data.proximity_mask[2]), np.sum(~Data.proximity_mask[3]))

def fraction(prox_mask):
    return np.sum(prox_mask)/(np.sum(prox_mask)+np.sum(~prox_mask))

print('Fraction In/Total MC events: ', fraction(MC.proximity_mask[0]), fraction(MC.proximity_mask[1]), fraction(MC.proximity_mask[2]), fraction(MC.proximity_mask[3]))
print('Fraction In/Total Data events: ', fraction(Data.proximity_mask[0]), fraction(Data.proximity_mask[1]), fraction(Data.proximity_mask[2]), fraction(Data.proximity_mask[3]))


Types = [Data, MC]

for i in range(4):
    bad_bins_filter = [value >= 0 for value in mcEvents['Ratio_Station_'+str(i)].to_list()]
    bad_bins = [k for k, x in enumerate(bad_bins_filter) if x]
    mc_bad_events = mcEvents[mcEvents['DEta_Bin_Station_'+str(i)].isin(bad_bins)]
    data_bad_events = dataEvents[dataEvents['DEta_Bin_Station_'+str(i)].isin(bad_bins)]

for type in Types:
    Muon_Eta = []
    Muon_Phi = []
    Event_Weight = []
    Muon_EtaN = []
    Muon_PhiN = []
    Event_WeightN = []
    
    if type == Data:
        EventWeight = data_bad_events['EventWeight']
        Muon_eta = data_bad_events['eta']
        Muon_phi = data_bad_events['phi']
    if type == MC:
        EventWeight = mc_bad_events['EventWeight']
        Muon_eta = mc_bad_events['eta']
        Muon_phi = mc_bad_events['phi']

    for i in range (4):
        Muon_Eta.append([float(value) for value in Muon_eta[type.proximity_mask[i]]]) #Muon_eta is eta masked
        Muon_Phi.append([float(value) for value in Muon_phi[type.proximity_mask[i]]]) #Muon_phi is eta masked
        Event_Weight.append([float(value) for value in EventWeight[type.proximity_mask[i]]])
        Muon_EtaN.append([float(value) for value in Muon_eta[~type.proximity_mask[i]]]) #Muon_eta is eta masked
        Muon_PhiN.append([float(value) for value in Muon_phi[~type.proximity_mask[i]]]) #Muon_phi is eta masked
        Event_WeightN.append([float(value) for value in EventWeight[~type.proximity_mask[i]]])


    Total_Hits = np.histogram2d(Muon_eta, Muon_phi, bins=(100,100), range=[[-2.5, 2.5],[-math.pi, math.pi]], density=None, weights=EventWeight)
    Prox_Hits = []
    Eff_Hits = []
    
    for i in range (4):
        Prox_Hits.append(np.histogram2d(Muon_Eta[i], Muon_Phi[i], bins=(100,100), range=[[-2.5, 2.5],[-math.pi, math.pi]], density=None, weights=Event_Weight[i]))
        Eff_Hits.append(np.divide(Prox_Hits[i][0],Total_Hits[0]))
        xedges, yedges = Prox_Hits[i][2], Prox_Hits[i][1]
        fig = plt.figure(figsize=(30, 30), dpi=100)
        ax = fig.add_subplot(131, title=str(type.name)+' Station '+str(i)+' DT hits, eta_mask + peak isolation DEta <= '+str(peak_lim))
        plt.imshow(Eff_Hits[i], interpolation='nearest', origin='lower',
            extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
        plt.xlabel('Muon $\phi$')
        plt.ylabel('Muon $\eta$')
        plt.colorbar(fraction=0.035, pad=0.04)
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
        
        plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)
        plt.tight_layout(pad=1.08, h_pad=None, w_pad=None, rect=None)
        plt.savefig(str(type.name)+' Station '+str(i)+' DT hits.jpg', bbox_inches='tight')
        plt.close()

for i in range(4):
    display(HTML("<table><tr><td><img src='data Station "+str(i)+" DT hits.jpg'></td><td><img src='mc Station "+str(i)+" DT hits.jpg'></td></tr></table>"))


NameError: name 'dataEvents' is not defined

In [ ]:
peak_lim = 0.25

Data.proximity_mask = []
MC.proximity_mask = []
for i in range(4):
    Data.proximity_mask.append(abs(data_bad_events['dtDEtabyStation_'+str(i)]) <= peak_lim)
    MC.proximity_mask.append(abs(mc_bad_events['dtDEtabyStation_'+str(i)])<= peak_lim)
    
print('In peak eta_masked MC events: ', np.sum(MC.proximity_mask[0]), np.sum(MC.proximity_mask[1]), np.sum(MC.proximity_mask[2]), np.sum(MC.proximity_mask[3]))
print('Out of peak eta_masked MC events: ', np.sum(~MC.proximity_mask[0]), np.sum(~MC.proximity_mask[1]), np.sum(~MC.proximity_mask[2]), np.sum(~MC.proximity_mask[3]))

print('In peak eta_masked Data events: ', np.sum(Data.proximity_mask[0]), np.sum(Data.proximity_mask[1]), np.sum(Data.proximity_mask[2]), np.sum(Data.proximity_mask[3]))
print('Out of peak eta_masked Data events: ', np.sum(~Data.proximity_mask[0]), np.sum(~Data.proximity_mask[1]), np.sum(~Data.proximity_mask[2]), np.sum(~Data.proximity_mask[3]))

def fraction(prox_mask):
    return np.sum(prox_mask)/(np.sum(prox_mask)+np.sum(~prox_mask))

print('Fraction In/Total eta_masked MC events: ', fraction(MC.proximity_mask[0]), fraction(MC.proximity_mask[1]), fraction(MC.proximity_mask[2]), fraction(MC.proximity_mask[3]))
print('Fraction In/Total eta_masked Data events: ', fraction(Data.proximity_mask[0]), fraction(Data.proximity_mask[1]), fraction(Data.proximity_mask[2]), fraction(Data.proximity_mask[3]))

In [ ]:
Data.proximity_mask = []
MC.proximity_mask = []

out_of_peak_lim = 1

for i in range(4):
    Data.proximity_mask.append(abs(data_bad_events['dtDEtabyStation_'+str(i)]) >= out_of_peak_lim)
    MC.proximity_mask.append(abs(mc_bad_events['dtDEtabyStation_'+str(i)]) >= out_of_peak_lim)

for type in Types:
    Muon_Eta = []
    Muon_Phi = []
    Event_Weight = []
    Muon_EtaN = []
    Muon_PhiN = []
    Event_WeightN = []
    
    if type == Data:
        EventWeight = data_bad_events['EventWeight']
        Muon_eta = data_bad_events['eta']
        Muon_phi = data_bad_events['phi']
    if type == MC:
        EventWeight = mc_bad_events['EventWeight']
        Muon_eta = mc_bad_events['eta']
        Muon_phi = mc_bad_events['phi']

    for i in range (4):
        Muon_Eta.append([float(value) for value in Muon_eta[type.proximity_mask[i]]]) #Muon_eta is eta masked
        Muon_Phi.append([float(value) for value in Muon_phi[type.proximity_mask[i]]]) #Muon_phi is eta masked
        Event_Weight.append([float(value) for value in EventWeight[type.proximity_mask[i]]])
        Muon_EtaN.append([float(value) for value in Muon_eta[~type.proximity_mask[i]]]) #Muon_eta is eta masked
        Muon_PhiN.append([float(value) for value in Muon_phi[~type.proximity_mask[i]]]) #Muon_phi is eta masked
        Event_WeightN.append([float(value) for value in EventWeight[~type.proximity_mask[i]]])


    Total_Hits = np.histogram2d(Muon_eta, Muon_phi, bins=(100,100), range=[[-2.5, 2.5],[-math.pi, math.pi]], density=None, weights=EventWeight)
    Prox_Hits = []
    Eff_Hits = []
    
    for i in range (4):
        Prox_Hits.append(np.histogram2d(Muon_Eta[i], Muon_Phi[i], bins=(100,100), range=[[-2.5, 2.5],[-math.pi, math.pi]], density=None, weights=Event_Weight[i]))
        Eff_Hits.append(np.divide(Prox_Hits[i][0],Total_Hits[0]))
        xedges, yedges = Prox_Hits[i][2], Prox_Hits[i][1]
        fig = plt.figure(figsize=(30, 30), dpi=100)
        ax = fig.add_subplot(131, title=str(type.name)+' Station '+str(i)+' DT hits, eta_mask + far isolation DEta >= '+str(out_of_peak_lim))
        plt.imshow(Eff_Hits[i], interpolation='nearest', origin='lower',
            extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
        plt.xlabel('Muon $\phi$')
        plt.ylabel('Muon $\eta$')
        plt.colorbar(fraction=0.035, pad=0.04)
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
        
        plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)
        plt.tight_layout(pad=1.08, h_pad=None, w_pad=None, rect=None)
        plt.savefig(str(type.name)+' Station '+str(i)+' DT hits_far.jpg', bbox_inches='tight')
        plt.close()

for i in range(4):
    display(HTML("<table><tr><td><img src='data Station "+str(i)+" DT hits_far.jpg'></td><td><img src='mc Station "+str(i)+" DT hits_far.jpg'></td></tr></table>"))


In [ ]:
weights = MC.EventWeight
for i in range(4):
    weights = weights*mcEvents['Ratio_Iter_'+str(i)].to_list()
    plt.hist(weights, bins=100, range=(0, 5), density=None)
    plt.xlabel('Iterated Ratio Weights')
    plt.ylabel('Freq')
    plt.show()

for i in range(4):
    weights = MC.EventWeight*mcEvents['Ratio_Station_'+str(i)].to_list()
    plt.hist(weights, bins=100, range=(0, 5), density=None)
    plt.xlabel('Ratio Weights')
    plt.ylabel('Freq')
    plt.show()

for i in range(4):
    plt.scatter(mcEvents['dtDRbyStation_'+str(i)].to_list(), mcEvents['EventWeight'].to_list())
    plt.xlabel('DR')
    plt.ylabel('Event Weight')
    plt.show()

for i in range(4):
    plt.scatter(mcEvents['dtDRbyStation_'+str(i)].to_list(), mcEvents['Ratio_Station_'+str(i)].to_list())
    plt.xlabel('DR')
    plt.ylabel('Ratio Weight')
    plt.show()